# Predict Relapse Rates
Use Random Forests and Gradient Boosted Forests (XGBoost) to predict targets which we believe IC Laws may influence, and evaluate the influence of IC Laws on the rate of relapse.  Evaluate models with and without state included as a predictor

### Import and prepare data

In [ ]:
## Load modules
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import joblib
from sklearn.model_selection import train_test_split as split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier as GBClass
from sklearn.ensemble import GradientBoostingClassifier as GBRreg


## Specify WD
wd = "/home/abf/BINF667013_Final_Project/"
#wd = (
#    "/Users/adam/Documents/BINF667013_BigDataAnalyticsHealthcare/" +
#    "Final_Project/TEDS_Study"
#)
os.chdir(wd)

## Load data
# teds_laws = pd.read_csv("analysis_objects/teds_laws.csv")
teds_imp_laws = pd.read_csv(
    "analysis_objects/teds_imp_laws.csv"
)

## Get index for patients that are either Complete Treatment or Drop Out
idx = (teds_imp_laws['REASON'] == 2) | (teds_imp_laws['REASON'] == 1)
teds_imp_laws = teds_imp_laws[idx].sample(5000)
print(teds_imp_laws.shape)

### Predict Dropout by Primary Substance, with IC Laws And State

In [ ]:
## Drop extraneous features
keep = ["IC_law", "alc_cases","alc_relapse"]
drop = teds_imp_laws.iloc[:,74:].columns.difference(keep).append(pd.Index([
    'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'SUB1_D','Jurisdiction',
    'SUB2_D', 'SUB3_D','METHFLG', 'OPSYNFLG', 'PCPFLG', 'FREQ1_D',
    'FREQ2_D','FREQ3_D','HALLFLG', 'MTHAMFLG', 'AMPHFLG','STIMFLG', 
    'BENZFLG', 'TRNQFLG','BARBFLG', 'SEDHPFLG','INHFLG', 'OTCFLG',
    'STATE','OTHERFLG','CBSA2010', 'REGION', 'DIVISION'
]))
data=teds_imp_laws.drop(columns=drop)

## Divide data into training and validation sets, stratified by alcohol relapse
alc_train, alc_test = split(
    data[data.alc_cases == 1], 
    train_size=0.7,
    stratify=data[data.alc_cases == 1].alc_relapse,
    random_state = 123
)

### Fit a Gradient Boosted Model to alcohol relapses. 
clf=GridSearchCV(
    GBClass(loss='deviance', random_state=123),
    param_grid={
        "learning_rate":[0.00001, 0.0001, 0.001, 0.01, 0.1],
        "n_estimators":[50, 100, 250, 500],
        "subsample":[0.8, 1]
    },
     n_jobs = -1,
    scoring="balanced_accuracy", 
    cv=5
)

model=clf.fit(
    X=alc_train.drop(columns=['REASON']),
    y=alc_train['REASON']
)


filename="analysis_objects/08_GB_Dropout_Classifier_Alc.sav"
joblib.dump(model, open(filename, "wb"))

train_pred=model.predict(alc_train.drop(columns=['REASON']))
print(
    confusion_matrix(alc_train['REASON'], train_pred)
)
print(classification_report(
    alc_train['REASON'], train_pred, zero_division=1,
    digits=3)
)

test_pred=model.predict(alc_test.drop(columns=['REASON']))
print(
    confusion_matrix(alc_test['REASON'], test_pred)
)
print(classification_report(
    alc_test['REASON'], test_pred, zero_division=1,
    digits=3)
)

## Extract and print parameters for best model
best = model.best_estimator_
print(best)

## Extract Importance and Rank Features
imp = best.feature_importances_
imp={i:imp[i] for i in range(0, len(imp))}
z=alc_train.drop(columns=['REASON']).columns.to_list()

sorted_imp = sorted(imp.items(), key=lambda kv: kv[1])
sorted_imp
importance = pd.DataFrame({
    "Feature": [z[i] for i in [i[0] for i in sorted_imp]],
    "Importance": [i[1] for i in sorted_imp]
})

#print(importance)
sns.set(rc = {'figure.figsize':(24,6)})
plt.xticks(rotation=90)
sns.barplot(x='Feature', y='Importance', data=importance)

In [ ]:
## Drop extraneous features
keep = ["IC_law", "hrn_cases","hrn_relapse"]
drop = teds_imp_laws.iloc[:,74:].columns.difference(keep).append(pd.Index([
    'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'SUB1_D','Jurisdiction',
    'SUB2_D', 'SUB3_D','METHFLG', 'OPSYNFLG', 'PCPFLG', 'FREQ1_D',
    'FREQ2_D','FREQ3_D','HALLFLG', 'MTHAMFLG', 'AMPHFLG','STIMFLG', 
    'BENZFLG', 'TRNQFLG','BARBFLG', 'SEDHPFLG','INHFLG', 'OTCFLG',
    'STATE','OTHERFLG','CBSA2010', 'REGION', 'DIVISION'
]))
data=teds_imp_laws.drop(columns=drop)

## Divide data into training and validation sets, stratified by alcohol relapse
hrn_train, hrn_test = split(
    data[data.hrn_cases == 1], 
    train_size=0.7,
    stratify=data[data.hrn_cases == 1].hrn_relapse,
    random_state = 123
)

### Fit a Gradient Boosted Model to alcohol relapses. 
clf=GridSearchCV(
    GBClass(loss='deviance', random_state=123),
    param_grid={
        "learning_rate":[0.00001, 0.0001, 0.001, 0.01, 0.1],
        "n_estimators":[50, 100, 250, 500],
        "subsample":[0.8, 1]
    },
     n_jobs = -1,
    scoring="balanced_accuracy", 
    cv=5
)

model=clf.fit(
    X=hrn_train.drop(columns=['REASON']),
    y=hrn_train['REASON']
)


filename="analysis_objects/08_GB_Dropout_Classifier_Hrn.sav"
joblib.dump(model, open(filename, "wb"))

train_pred=model.predict(hrn_train.drop(columns=['REASON']))
print(
    confusion_matrix(hrn_train['REASON'], train_pred)
)
print(classification_report(
    hrn_train['REASON'], train_pred, zero_division=1,
    digits=3)
)

test_pred=model.predict(hrn_test.drop(columns=['REASON']))
print(
    confusion_matrix(hrn_test['REASON'], test_pred)
)
print(classification_report(
    hrn_test['REASON'], test_pred, zero_division=1,
    digits=3)
)

## Extract and print parameters for best model
best = model.best_estimator_
print(best)

## Extract Importance and Rank Features
imp = best.feature_importances_
imp={i:imp[i] for i in range(0, len(imp))}
z=hrn_train.drop(columns=['REASON']).columns.to_list()

sorted_imp = sorted(imp.items(), key=lambda kv: kv[1])
sorted_imp
importance = pd.DataFrame({
    "Feature": [z[i] for i in [i[0] for i in sorted_imp]],
    "Importance": [i[1] for i in sorted_imp]
})

#print(importance)
sns.set(rc = {'figure.figsize':(24,6)})
plt.xticks(rotation=90)
sns.barplot(x='Feature', y='Importance', data=importance)

In [ ]:
## Drop extraneous features
keep = ["IC_law", "met_cases","met_relapse"]
drop = teds_imp_laws.iloc[:,74:].columns.difference(keep).append(pd.Index([
    'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'SUB1_D','Jurisdiction',
    'SUB2_D', 'SUB3_D','METHFLG', 'OPSYNFLG', 'PCPFLG', 'FREQ1_D',
    'FREQ2_D','FREQ3_D','HALLFLG', 'MTHAMFLG', 'AMPHFLG','STIMFLG', 
    'BENZFLG', 'TRNQFLG','BARBFLG', 'SEDHPFLG','INHFLG', 'OTCFLG',
    'STATE','OTHERFLG','CBSA2010', 'REGION', 'DIVISION'
]))
data=teds_imp_laws.drop(columns=drop)

## Divide data into training and validation sets, stratified by alcohol relapse
met_train, met_test = split(
    data[data.met_cases == 1], 
    train_size=0.7,
    stratify=data[data.met_cases == 1].met_relapse,
    random_state = 123
)

### Fit a Gradient Boosted Model to alcohol relapses. 
clf=GridSearchCV(
    GBClass(loss='deviance', random_state=123),
    param_grid={
        "learning_rate":[0.00001, 0.0001, 0.001, 0.01, 0.1],
        "n_estimators":[50, 100, 250, 500],
        "subsample":[0.8, 1]
    },
     n_jobs = -1,
    scoring="balanced_accuracy", 
    cv=5
)

model=clf.fit(
    X=met_train.drop(columns=['REASON']),
    y=met_train['REASON']
)


filename="analysis_objects/08_GB_Dropout_Classifier_Met.sav"
joblib.dump(model, open(filename, "wb"))

train_pred=model.predict(met_train.drop(columns=['REASON']))
print(
    confusion_matrix(met_train['REASON'], train_pred)
)
print(classification_report(
    met_train['REASON'], train_pred, zero_division=1,
    digits=3)
)

test_pred=model.predict(met_test.drop(columns=['REASON']))
print(
    confusion_matrix(met_test['REASON'], test_pred)
)
print(classification_report(
    met_test['REASON'], test_pred, zero_division=1,
    digits=3)
)

## Extract and print parameters for best model
best = model.best_estimator_
print(best)

## Extract Importance and Rank Features
imp = best.feature_importances_
imp={i:imp[i] for i in range(0, len(imp))}
z=met_train.drop(columns=['REASON']).columns.to_list()

sorted_imp = sorted(imp.items(), key=lambda kv: kv[1])
sorted_imp
importance = pd.DataFrame({
    "Feature": [z[i] for i in [i[0] for i in sorted_imp]],
    "Importance": [i[1] for i in sorted_imp]
})

#print(importance)
sns.set(rc = {'figure.figsize':(24,6)})
plt.xticks(rotation=90)
sns.barplot(x='Feature', y='Importance', data=importance)